# Strings
- A string is like a special kind of array but is immutable

## Tips
- Similar to arrays, string problems often have simple brute-force solutions that use $O(n)$ space, but subtler solutions that use the string itself to **reduce the complexit** to $O(1)$
- Understand the **implications** of a string type which is **immutable**, e.g., the need to allocate a new string when concatenating immutable strings. Know **alternatives** to immutable strings, e.g., a **list** in Python
- Updating a mutable string from the font is slow, so see if it's possible to **write values from the back**
- indexing works the same as lists

In [39]:
from typing import List, Iterator, Tuple
import bisect
import collections
import math
import functools
import random

from utils import run_tests

## Libraries

In [14]:
s = 'The cow jumped over the moon'
t = 'The moon is made of cheese'
print(s)
print(t)

print('\ns.startswith("The"):       ', s.startswith("The"))

print('\ns.endswith(("moo", "moon"):', s.endswith(("moo", "moon")))     # tuple of string to try

print('\ns + t:                     ', s + t)

strings = ['the', 'cat', 'and', 'the', 'hat']
print('\n', strings)
print('" ".join(strings):           ' , " ".join(strings))

The cow jumped over the moon
The moon is made of cheese

s.startswith("The"):        True

s.endswith(("moo", "moon"): True

s + t:                      The cow jumped over the moonThe moon is made of cheese

 ['the', 'cat', 'and', 'the', 'hat']
" ".join(strings):            the cat and the hat


### Is Palindrome?
A palindrome is a string the reads the same forwards and backwards.  
Key to optimal solution is to traverse string forward and backwards to simultaneously

In [17]:
def is_palindrome(s: str) -> bool:
    # note: ~i = -(i+1)
    return all(s[i] == s[~i] for i in range(len(s) // 2))

inputs, outputs = ('cat', 'aabbaa', 'aba', 'abca'), (False, True, True, False)
run_tests(is_palindrome, inputs, outputs)

Time complexity is $O(n)$ and space complexity $O(1)$

### Interconvert Strings and Integers

In [43]:
def int_to_string(num: int) -> str:

    if num < 0:
        is_negative, num = True, abs(num)
    else:
        is_negative = False
    
    digits = []
    # process one digit at a time
    # processing digits in reverse order
    while True:
        num, digit = num // 10, num % 10
        digits.append(chr(ord('0') + digit))   # get code 0, add digit, then convert to character
        if num == 0:
            break

    if is_negative:
        digits.append('-')

    return ''.join(reversed(digits))


# sould be able to handle '314', '+314' or '-314'
def string_to_int(s: str) -> int:
    string_digits = {s:d for s, d, in zip(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], range(10))}

    sign = -1 if s[0] == '-' else 1

    running_sum = 0
    for i in s[s[0] in '+-':]:             # this skips first entry if has symbol
        running_sum = running_sum * 10 + string_digits[i]  # mutliplying by 10 shift place value to left

    return sign * running_sum


def string_to_int_v2(s: str) -> int:
    string_digits = {s:d for s, d, in zip(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], range(10))}
    return (-1 if s[0] == '-' else 1) * functools.reduce(
        lambda running_sum, c: running_sum * 10 + string_digits[c],
        s[s[0] in '+-':], 0
    )

inputs, outputs = (123, -123, 4, -4, 0), ('123', '-123', '4', '-4', '0')
run_tests(int_to_string, inputs, outputs)

inputs, outputs = ('123', '-123', '+123', '4', '-4', '+4', '0'), (123, -123, 123, 4, -4, 4, 0)
run_tests(string_to_int, inputs, outputs)

run_tests(string_to_int_v2, inputs, outputs)

In [28]:
print('ord("0") - returns Unicode code point one-character string,      e.g.:',  ord('0'))
print('chr(ord("0")) - returns Unicode string for one-character string, e.g.:',  chr(ord('0')))
print('ord("0") + 5:     ',  ord('0') + 5)
print('chr(ord("0") + 5):',  chr(ord('0') + 5))


ord("0") - returns Unicode code point one-character string,      e.g.: 48
chr(ord("0")) - returns Unicode string for one-character string, e.g.: 0
ord("0") + 5:      53
chr(ord("0") + 5): 5


In [31]:
{s:d for s, d, in zip(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], range(10))}

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9}

### Base Conversion:
